# Vanishing Point Detection  
**Canny + Probabilistic Hough Transform + RANSAC**  

Classical but still very effective approach for one-point perspective scenes  
(roads, railways, corridors, architecture, etc.)  

**Pipeline overview:**  
1. Grayscale + strong edge detection (Canny)  
2. Line segment detection (HoughLinesP)  
3. Robust vanishing point estimation via **RANSAC** on line intersections  

**2025 update notes:**  
Still very useful when you need:  
• interpretable results  
• low compute / no deep learning  
• good initialization for SLAM / calibration / 3D reconstruction

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

## 1. Load & Prepare Image

In [ ]:
# You can replace with any road / corridor / building interior image
IMAGE_PATHS = [
    "road.jpg",                    # your local file
    # "https://i.ibb.co/yWmY6cz/road.jpg",
    # "https://images.unsplash.com/photo-1506905925346-21bda4d32df4",  # mountain road
    # "https://images.unsplash.com/photo-1511893736272-2817e2d3e462",  # long corridor
]

# For demo we'll assume you have at least one local image
img_path = IMAGE_PATHS[0]

if not os.path.exists(img_path):
    print("Please download an example road image and save it as 'road.jpg'")
    print("Recommended quick sources:")
    print("  • https://i.ibb.co/yWmY6cz/road.jpg")
    print("  • https://unsplash.com/photos/gray-concrete-road-between-green-trees-during-daytime-7u5Mw2H7k2Q")
else:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    plt.figure(figsize=(12,6))
    plt.subplot(121), plt.imshow(img), plt.title('Original')
    plt.subplot(122), plt.imshow(gray, cmap='gray'), plt.title('Grayscale')
    plt.tight_layout()
    plt.show()

## 2. Edge Detection (Canny)

In [ ]:
# ── Most important parameters ───────────────────────────────────────
CANNY_LOW    = 70
CANNY_HIGH   = 180
CANNY_APERTURE = 3

edges = cv2.Canny(
    gray,
    threshold1 = CANNY_LOW,
    threshold2 = CANNY_HIGH,
    apertureSize = CANNY_APERTURE,
    L2gradient = True
)

plt.figure(figsize=(10,10))
plt.imshow(edges, cmap='gray')
plt.title('Canny Edges')
plt.axis('off')
plt.show()

## 3. Line Detection – Probabilistic Hough Transform

In [ ]:
# ── HoughLinesP parameters ──────────────────────────────────────────
HOUGH_RHO         = 1
HOUGH_THETA       = np.pi / 180
HOUGH_THRESHOLD   = 90      # min votes
MIN_LINE_LENGTH   = 100
MAX_LINE_GAP      = 15

lines = cv2.HoughLinesP(
    edges,
    rho        = HOUGH_RHO,
    theta      = HOUGH_THETA,
    threshold  = HOUGH_THRESHOLD,
    minLineLength = MIN_LINE_LENGTH,
    maxLineGap    = MAX_LINE_GAP
)

print(f"Detected {len(lines) if lines is not None else 0} line segments")

# Visualization
line_img = img.copy()
if lines is not None:
    for [[x1,y1,x2,y2]] in lines:
        cv2.line(line_img, (x1,y1), (x2,y2), (0, 255, 100), 2, cv2.LINE_AA)

plt.figure(figsize=(12,8))
plt.imshow(line_img)
plt.title('Detected Line Segments')
plt.axis('off')
plt.show()

## 4. RANSAC-based Vanishing Point Estimation

In [ ]:
def line_to_homogeneous(x1,y1,x2,y2):
    """ Line in homogeneous form: ax + by + c = 0 """
    a = y1 - y2
    b = x2 - x1
    c = x1*y2 - x2*y1
    return np.array([a, b, c], dtype=float)


def intersection(l1, l2, eps=1e-8):
    """ Intersection point of two lines in homogeneous coordinates """
    p = np.cross(l1, l2)
    if abs(p[2]) < eps:
        return None  # parallel / same line
    return p[:2] / p[2]


def point_line_distance(point, line):
    """ Distance from point to line (ax+by+c=0) """
    x, y = point
    a, b, c = line
    return abs(a*x + b*y + c) / np.sqrt(a*a + b*b + 1e-10)


# Prepare lines in normal form
if lines is None:
    print("No lines detected - cannot continue")
else:
    lines_h = [line_to_homogeneous(x1,y1,x2,y2) for [[x1,y1,x2,y2]] in lines]

    best_vp = None
    best_inliers = 0
    best_mask = None

    N_ITERATIONS   = 1200
    INLIER_TH      = 4.0       # pixels
    MIN_INLIERS    = 15

    n_lines = len(lines_h)

    for _ in range(N_ITERATIONS):
        idx = np.random.choice(n_lines, 2, replace=False)
        l1, l2 = lines_h[idx[0]], lines_h[idx[1]]

        vp_candidate = intersection(l1, l2)
        if vp_candidate is None:
            continue

        # Count inliers
        distances = [point_line_distance(vp_candidate, l) for l in lines_h]
        inliers = np.array(distances) < INLIER_TH
        count = np.sum(inliers)

        if count > best_inliers:
            best_inliers = count
            best_vp = vp_candidate
            best_mask = inliers

    print(f"Best model found with {best_inliers} inliers ({best_inliers/n_lines:.1%})")

    # ── Visualization ───────────────────────────────────────────────
    result = img.copy()

    # Draw all lines (faint)
    for i, [[x1,y1,x2,y2]] in enumerate(lines):
        color = (80,80,255) if not best_mask[i] else (0,220,50)
        cv2.line(result, (x1,y1), (x2,y2), color, 2, cv2.LINE_AA)

    if best_vp is not None:
        x, y = map(int, best_vp)
        # Draw big red vanishing point
        cv2.circle(result, (x,y), 12, (255,30,30), -1, cv2.LINE_AA)
        cv2.circle(result, (x,y), 18, (255,255,255), 3, cv2.LINE_AA)

        # Optional: draw lines going through VP (for debug)
        # for l in lines_h:
        #     if point_line_distance(best_vp, l) < INLIER_TH*1.5:
        #         pt = intersection(l, np.array([0,0,1]))  # dummy
        #         if pt is not None:
        #             cv2.line(result, (x,y), tuple(map(int,pt)), (200,200,255), 1)

    plt.figure(figsize=(14,10))
    plt.imshow(result)
    plt.title(f'Vanishing Point (RANSAC)  –  {best_inliers} inliers')
    plt.axis('off')
    plt.show()

## Quick tuning guide

| Stage          | Most sensitive parameters                          | Typical range / advice                             |
|---------------|----------------------------------------------------|-----------------------------------------------------|
| Canny         | low/high threshold                                 | low: 50–120, high: 120–250                         |
| HoughP        | threshold, minLineLength, maxLineGap               | threshold 70–140, minLen 60–180, gap 8–25          |
| RANSAC        | inlier threshold (px), nb iterations               | 3–8 pixels, 800–2500 iterations                    |

Happy vanishing point hunting! 🚗